<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/analyse_NY_mobility_poverty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
!pip install ipython-autotime

%load_ext autotime

time: 1.97 ms (started: 2021-04-05 03:36:35 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 40.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=fae137f910385916cdeb5d3a5fc9cd8b048975989a56c3619712281a1c9db6a0
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 195 ms (started: 2021-04-05 04:03:40 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 332 ms (started: 2021-04-05 04:03:42 +00:00)


In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 5.48 s (started: 2021-04-05 04:03:43 +00:00)


In [6]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 19.7 s (started: 2021-04-05 04:03:49 +00:00)


In [7]:
# Demographic Data Chicago
!ls drive/MyDrive/big-data-project/data/clean-data/ny/ny.csv

!ls drive/MyDrive/big-data-project/data/clean-data/ny/social/2020/



drive/MyDrive/big-data-project/data/clean-data/ny/poverty_and_white_percentage.csv
2020-01-01-social-distancing.csv  2020-07-02-social-distancing.csv
2020-01-02-social-distancing.csv  2020-07-03-social-distancing.csv
2020-01-03-social-distancing.csv  2020-07-04-social-distancing.csv
2020-01-04-social-distancing.csv  2020-07-05-social-distancing.csv
2020-01-05-social-distancing.csv  2020-07-06-social-distancing.csv
2020-01-06-social-distancing.csv  2020-07-07-social-distancing.csv
2020-01-07-social-distancing.csv  2020-07-08-social-distancing.csv
2020-01-08-social-distancing.csv  2020-07-09-social-distancing.csv
2020-01-09-social-distancing.csv  2020-07-10-social-distancing.csv
2020-01-10-social-distancing.csv  2020-07-11-social-distancing.csv
2020-01-11-social-distancing.csv  2020-07-12-social-distancing.csv
2020-01-12-social-distancing.csv  2020-07-13-social-distancing.csv
2020-01-13-social-distancing.csv  2020-07-14-social-distancing.csv
2020-01-14-social-distancing.csv  2020-07-15-s

In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 3.5 ms (started: 2021-04-05 04:04:35 +00:00)


In [9]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [10]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [ ]:
# # Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 2.72 ms (started: 2021-04-04 21:58:00 +00:00)


In [11]:
df_soc.createOrReplaceTempView('T')
df_soc = spark.sql('SELECT cbg, date_range_start, device_count, completely_home_device_count FROM T')
df_soc.show()

+------------+--------------------+------------+----------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|
+------------+--------------------+------------+----------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|
|360810384001|2020-02-03T00:00:...|          84|                          21|
|360850170103|2020-02-03T00:00:...|         250|                          68|
|360050213021|2020-02-03T00:00:...|          92|                          26|
|360050253004|2020-02-03T00:00:...|          52|                          13|
|360470385003|2020-02-03T00:00:...|          37|                           9|
|360470406002|2020-02-03T00:00:...|          46|                          12|
|360470428003|2020-02-03T00:00:...|          84|                          20|
|360471130002|2020-02-03T00:00:...|          84|                          20|
|360811029001|2020-02-03T00:00:...|         256|                

In [12]:
# View schema
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)

time: 3.98 ms (started: 2021-04-05 04:05:08 +00:00)


In [13]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc.show()

+------------+--------------------+------------+----------------------------+--------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|completely_home_percentage|
+------------+--------------------+------------+----------------------------+--------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                    28.125|
|360810384001|2020-02-03T00:00:...|          84|                          21|                      25.0|
|360850170103|2020-02-03T00:00:...|         250|                          68|        27.200000000000003|
|360050213021|2020-02-03T00:00:...|          92|                          26|         28.26086956521739|
|360050253004|2020-02-03T00:00:...|          52|                          13|                      25.0|
|360470385003|2020-02-03T00:00:...|          37|                           9|        24.324324324324326|
|360470406002|2020-02-03T00:00:...|          46|       

In [14]:
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)

time: 2.52 ms (started: 2021-04-05 04:05:17 +00:00)


In [15]:
df_soc.createOrReplaceTempView('t')
# get mean percentage for each cbg
df_soc = spark.sql('SELECT cbg, AVG(completely_home_percentage) AS mean_completely_home_percentage FROM t GROUP BY cbg')
df_soc.createOrReplaceTempView('t')
df_soc.show()

+------------+-------------------------------+
|         cbg|mean_completely_home_percentage|
+------------+-------------------------------+
|360610070005|              26.74619160062064|
|360050181024|              42.62318749728753|
|360810152002|               38.8657049221352|
|360470800001|              33.01453293021799|
|360610063005|             29.345621721819686|
|360810278002|              41.37884085368488|
|360470003011|             29.189053595079148|
|360050076001|              42.32023402808608|
|360810034002|              48.57073496460549|
|360050090001|             21.241432500320673|
|360470828001|              34.16920415277364|
|360850169012|               40.6464671712979|
|360050157003|              41.53771715243185|
|360810757022|              38.35549451513402|
|360810838003|              35.36659337984723|
|360050413007|              40.57397978322256|
|360050393006|              43.79182648318182|
|360810040012|              34.34259697132889|
|360610048004

In [16]:
# Read poverty data
# Read data into dataframe
df_pov = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ny.csv'))
df_pov.createOrReplaceTempView('d')
df_pov.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|360050175002|     1110| 55.22522522522523|14.774774774774773|
|360050141001|     1445| 36.74740484429066| 19.16955017301038|
|360050145001|     2003| 39.47667804323095|  8.53719420868697|
|360050075002|     2731|22.812156719150494|4.5404613694617355|
|360050418001|      936|18.859649122807017| 9.081196581196581|
|360050125001|     2935| 43.24693042291951|12.742759795570699|
|360050127012|     1061| 43.73232799245994|18.378887841658813|
|360050054003|     1031| 57.12900096993211| 28.12803103782735|
|360050027011|     1364|  65.9090909090909|14.589442815249267|
|360050458001|     1168| 37.15753424657534| 2.910958904109589|
|360050224011|     1291|19.907048799380327|27.420604182804027|
|360050389001|     1143| 53.94265232974911|30.708661417322837|
|360050071002|     1661|           37.4375|11.499096929

In [17]:
# join mobility and poverty
result = spark.sql('SELECT d.*, t.mean_completely_home_percentage FROM t INNER JOIN d ON t.cbg = d.cbg')
result.createOrReplaceTempView('t')
result.show()

+------------+---------+------------------+------------------+-------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|
+------------+---------+------------------+------------------+-------------------------------+
|360610070005|     1771|17.554125219426563| 66.00790513833992|              26.74619160062064|
|360050181024|     1457|14.070006863417984| 6.314344543582704|              42.62318749728753|
|360810152002|     1386| 16.81096681096681|26.767676767676768|               38.8657049221352|
|360470800001|      812|27.832512315270936| 38.42364532019704|              33.01453293021799|
|360610063005|      884| 4.046242774566474| 85.85972850678732|             29.345621721819686|
|360810278002|     1796|32.962138084632514| 4.844097995545657|              41.37884085368488|
|360470003011|      661| 9.937888198757763| 77.60968229954615|             29.189053595079148|
|360050076001|      631|27.403846153846157| 9.6671

In [18]:
# Count number of rows
spark.sql('SELECT COUNT(cbg) FROM t').show()

+----------+
|count(cbg)|
+----------+
|      6369|
+----------+

time: 37 s (started: 2021-04-05 04:31:50 +00:00)


In [19]:
# sort by poverty
result = spark.sql('SELECT * FROM t ORDER BY CAST(poverty_percentage AS float) DESC')
result.createOrReplaceTempView('t')
result.show()

+------------+---------+------------------+------------------+-------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|
+------------+---------+------------------+------------------+-------------------------------+
|360610240001|     1742| 89.67653131452168|36.337543053960964|             24.711135516948634|
|360810849004|      537| 87.33705772811918|4.8417132216014895|             33.971280051419555|
|360470910002|     1588| 80.96166341780376| 5.289672544080604|              41.00827096908293|
|360470535004|     1373|  78.6598689002185| 94.31900946831755|              29.56160175262152|
|360050020003|     3073| 76.88787185354691|42.629352424341036|              43.17715123953364|
|360050399022|      732|              75.0| 5.327868852459016|               36.6849700226435|
|360610143001|        5|              75.0|              40.0|              41.51945722123565|
|360050391006|     1134| 74.33862433862434|15.9611

In [ ]:
# group by poverty range


time: 1.86 ms (started: 2021-04-04 22:10:28 +00:00)


In [20]:
result_df = result.toPandas()
result_df

,cbg,pop_total,poverty_percentage,perc_whiteonly,mean_completely_home_percentage
0,360610240001,1742,89.67653131452168,36.337543053960964,24.711136
1,360810849004,537,87.33705772811918,4.8417132216014895,33.971280
2,360470910002,1588,80.96166341780376,5.289672544080604,41.008271
3,360470535004,1373,78.6598689002185,94.31900946831755,29.561602
4,360050020003,3073,76.88787185354691,42.629352424341036,43.177151
...,...,...,...,...,...
6364,360470890003,468,0.0,0.0,33.685275
6365,360610086010,0,0.0,0.0,39.417101
6366,360810455006,0,0.0,0.0,32.755063
6367,360610173003,1470,0.0,67.9591836734694,38.309518


time: 43 s (started: 2021-04-05 04:43:15 +00:00)


In [21]:
# plot bar graph
alt.Chart(result_df).mark_point().encode(
    alt.X('poverty_percentage:Q'),
    alt.Y('mean_completely_home_percentage:Q'),
    tooltip=['cbg','poverty_percentage','mean_completely_home_percentage']
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 203 ms (started: 2021-04-05 04:44:02 +00:00)


In [22]:
temp = result_df.groupby(pd.cut(result_df['poverty_percentage'].astype(float), np.arange(0,110, 10))).mean().reset_index()

time: 47.7 ms (started: 2021-04-05 04:44:06 +00:00)


In [23]:
temp['poverty_percentage'] = temp['poverty_percentage'].astype(str)

time: 8.3 ms (started: 2021-04-05 04:44:08 +00:00)


In [24]:
temp['color'] = ['#45a0d1' if x>30 else '#ff4833' for x in temp['mean_completely_home_percentage']]
temp

,poverty_percentage,mean_completely_home_percentage,color
0,"(0, 10]",36.297662,#45a0d1
1,"(10, 20]",38.616430,#45a0d1
2,"(20, 30]",39.526510,#45a0d1
3,"(30, 40]",40.109901,#45a0d1
4,"(40, 50]",39.939022,#45a0d1
5,"(50, 60]",39.033690,#45a0d1
6,"(60, 70]",38.482398,#45a0d1
7,"(70, 80]",36.864316,#45a0d1
8,"(80, 90]",33.230229,#45a0d1
9,"(90, 100]",NaN,#ff4833


time: 24.6 ms (started: 2021-04-05 04:44:09 +00:00)


In [25]:
# 
alt.Chart(temp).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_completely_home_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_completely_home_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 24 ms (started: 2021-04-05 04:44:11 +00:00)


In [26]:
result_df['poverty_percentage'].astype(float).dtype

dtype('float64')

time: 6.9 ms (started: 2021-04-05 04:44:15 +00:00)
